# Reproduce Latents

In [13]:
%matplotlib inline

from importlib import reload
import os
import numpy as np
from pkg_resources import resource_filename

import pandas

import h5py
import seaborn as sns
import pandas as pd
import umap
from ulmo import io as ulmo_io
from ulmo import plotting
from ulmo.ssl.my_util import modis_loader, set_model
from ulmo.ssl.my_util import Params, option_preprocess
from ulmo.ssl import latents_extraction

import torch

from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from matplotlib import pyplot as plt
from tqdm import trange

# Load files

## Option file

In [2]:
# Load options
opt_file = os.path.join(resource_filename('ulmo', 'runs'),
                        'SSL', 'First','experiments', 
                        'base_modis_model', 'opts.json')

In [3]:
opt = Params(opt_file)
opt = option_preprocess(opt)

## Pytorch file

In [3]:
s3_model_path = 's3://modis-l2/modis_simclr_base_model/SimCLR_modis_resnet50_lr_0.05_decay_0.0001_bsz_64_temp_0.07_trial_0_cosine_warm/last.pth'
ulmo_io.download_file_from_s3(os.path.basename(s3_model_path), s3_model_path)

Done!


## Erdong's reproduced

In [6]:
s3_reproduce_latents = 's3://modis-l2/modis_latents_last_reproduced.h5'
ulmo_io.download_file_from_s3(os.path.basename(s3_reproduce_latents), s3_reproduce_latents)

Done!


In [34]:
with h5py.File('modis_latents_last_reproduced.h5', 'r') as f:
    reproduced = f['modis_latents'][:]
reproduced.shape

(134162, 128)

## Load one image

In [25]:
modis_dataset_path = "s3://modis-l2/PreProc/MODIS_2010_95clear_128x128_inpaintT_preproc_0.8valid.h5"
with ulmo_io.open(modis_dataset_path, 'rb') as f:
    hf = h5py.File(f, 'r')
    img0 = hf['train'][0:1,...]

In [26]:
img0.shape

(1, 1, 64, 64)

# Defs

## Load model

In [4]:
model_path = './'
model_name = "last.pth"
model_path_title = os.path.join(model_path, model_name)

In [19]:
def load_model(opt, model_path, remove_module=True):
    using_gpu = torch.cuda.is_available()
    model, _ = set_model(opt, cuda_use=using_gpu)
    if not using_gpu:
        model_dict = torch.load(model_path, map_location=torch.device('cpu'))
    else:
        model_dict = torch.load(model_path)

    if remove_module:
        new_dict = {}
        for key in model_dict['model'].keys():
            new_dict[key.replace('module.','')] = model_dict['model'][key]
        model.load_state_dict(new_dict)
    else:
        model.load_state_dict(model_dict['model'])
    print("Model loaded")
    return model

## Calculate latents

In [30]:
def calc_latent(model, img, using_gpu):
    modis_data = np.repeat(img, 3, axis=1)
    image_tensor = torch.tensor(modis_data)
    latents_np = latents_extraction.calc_latent(model, image_tensor, using_gpu)
    return latents_np

# Run on CPU

In [16]:
using_gpu = False

## Load model

In [20]:
cpu_model = load_model(opt, model_path_title, remove_module=True)

Model loaded


## Calc latents

In [31]:
with torch.no_grad():
    cpu_latents = calc_latent(cpu_model, img0, using_gpu)

In [32]:
cpu_latents

array([[-0.02050127,  0.07407618,  0.04224342, -0.0722819 ,  0.00294061,
         0.01800058,  0.12333521, -0.04895734, -0.03287374,  0.08677723,
         0.07657994, -0.05697816, -0.05136307, -0.08338682, -0.00944599,
        -0.12602489,  0.0230847 , -0.04223683,  0.0372562 , -0.04805676,
        -0.10619931, -0.13996214,  0.08047187,  0.13067569, -0.13676985,
         0.17556271, -0.05703831,  0.06767135, -0.14134355,  0.12577687,
        -0.07186496, -0.05439653, -0.10089992,  0.0928174 , -0.02457148,
        -0.05033816,  0.03225619, -0.11719637, -0.11773968,  0.01282729,
        -0.00686526, -0.07831512, -0.03915553,  0.0032813 ,  0.1243174 ,
        -0.05613922, -0.08427165,  0.11099236,  0.0152957 ,  0.13387296,
         0.09926886,  0.05570457,  0.06872411,  0.02095909, -0.0634723 ,
        -0.03160271, -0.0385897 , -0.025409  ,  0.05674249, -0.05657056,
         0.07395758, -0.04900097, -0.12068846,  0.04964582, -0.11162041,
         0.03284034,  0.07940092,  0.05017281,  0.1

# Run on 2 GPU

In [37]:
using_gpu = True

In [ ]:
gpu_model = load_model(opt, model_path_title, remove_module=False)